In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os
uri = os.environ.get('URI')

In [2]:
# obtaining the data
url = 'https://api.blockchain.info/charts/transactions-per-second?timespan=all&sampled=false&metadata=false&cors=true&format=json'
resp = requests.get(url)
data = pd.DataFrame(resp.json()['values'])

In [3]:
# parsing the date
# X - datetiem in UTC
# y - number of transactions added to the pool per minute
data['x'] = [datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in data['x']]
data['x'] = pd.to_datetime(data['x'])

In [4]:
data['x_hour'] = data['x'].dt.round('H')

C:\Users\Z\AppData\Local\Temp\ipykernel_9200\3266891699.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data['x_hour'] = data['x'].dt.round('H')


In [5]:
data_grouped = data.groupby('x_hour').sum(['y']).reset_index()

In [6]:
new_column_names = ['datetime', 'trans_per_min']
data_grouped.columns=new_column_names

In [7]:
yesterday_date = datetime.now() - timedelta(days=1)
new_data = data_grouped.loc[data_grouped['datetime'] >= yesterday_date,:]

In [39]:
new_file_name = 'etl_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.csv'
new_file_name = new_file_name.replace('-', '_')
new_file_path = os.path.join('data', new_file_name)

new_data.to_csv(new_file_path)

'etl_20240610_133756.csv'